<a href="https://colab.research.google.com/github/iued-uni-heidelberg/DAAD-Training-2021/blob/main/Terminologieextraktion5KeynessScoresV01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The analysis of keyness, which will be used in term extraction

---
We will also prepare the gold standard evaluation set for the project


In [ ]:
# Stage 0: Preparing Gold standard: Reading / extracting information from gold standard: creating a list of annotated terms
!wget https://heibox.uni-heidelberg.de/f/ae1110c4f9ad42b9a3d5/?dl=1
!mv index.html?dl=1 BGH1_s00Astghik.txt
!wget https://heibox.uni-heidelberg.de/f/0c787f26123f49178639/?dl=1
!mv index.html?dl=1 BGH1_s00Hayk.txt

In [ ]:
!cat BGH1_s00Astghik.txt BGH1_s00Hayk.txt >BGH1_s00GoldStandard.txt

In [ ]:
FInput = open('BGH1_s00GoldStandard.txt', 'r')
FOutput = open('BGH1_s00GoldS_Terms.txt', 'w')
# for statistical purposes - separately single and multiword terms
FOutputDict1w = open('BGH1_s00GoldS_D1w.txt', 'w') # 1-word terms
FOutputDict2w = open('BGH1_s00GoldS_D2w.txt', 'w') # 2-word terminological expressions
FOutputDictMWE = open('BGH1_s00GoldS_DMWE.txt', 'w') # more than 2 words

In [ ]:
def printDictionary(DictionaryFrq, FOut, Rev = True):
    for Word, Frq in sorted( DictionaryFrq.items() , key=lambda x: x[1], reverse=Rev):
        FOut.write(Word + '\t' + str(Frq) + '\n')
    return

In [ ]:
import re, os, sys
LGSTerms = [] # gold standard terms
DGS1w = {} # dictionary of single words
DGSMWE = {} # dictionary of mwes
IGS1w = 0 # number of annotated tokens of single words
IGSMWE = 0 # number of annotated tokens of multiwords
for SLine in FInput:
    LAnnotatedTermsInLine = re.findall('<<([^><]+)>>', SLine)
    LGSTerms.extend(LAnnotatedTermsInLine)

for GSTerm in LGSTerms:
    GSTerm = GSTerm.strip()
    GSTerm = re.sub(' +', ' ', GSTerm)
    LGSTErms = re.split(' ', GSTerm)
    if len(LGSTErms) > 2:
        IGSMWE += 1
        try: DGSMWE[GSTerm] += 1
        except: DGSMWE[GSTerm] = 1
    if len(LGSTErms) == 2:

    else:
        IGS1w += 1
        try: DGS1w[GSTerm] +=1
        except: DGS1w[GSTerm] = 1

    FOutput.write(GSTerm + '\n')

FOutputDictMWE.write('# Number of tokens: ' + str(IGSMWE) + '\n')
FOutputDict1w.write('# Number of tokens: ' + str(IGS1w) + '\n')

printDictionary(DGSMWE, FOutputDictMWE)
printDictionary(DGS1w, FOutputDict1w)

FOutputDictMWE.flush()
FOutputDictMWE.close()
FOutputDict1w.flush()
FOutputDict1w.close()

FOutput.flush()
FOutput.close()

FInput.close()